<a href="https://colab.research.google.com/github/felixalguzman/AmazonFinal/blob/master/keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc2


In [1]:
import spacy.cli
spacy.cli.download("es_core_news_sm")


✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


In [3]:
import spacy

# Loads the spacy en model into a python object
nlp = spacy.load('es_core_news_sm')


def separar_oracion_tokens(oracion: str):
    return [token.text for token in nlp(oracion)]


print('tokens? {}'.format(separar_oracion_tokens('Qué es el contrato de trabajo?')))

tokens? ['Qué', 'es', 'el', 'contrato', 'de', 'trabajo', '?']


In [4]:
!pip install gensim
from gensim.models.keyedvectors import  KeyedVectors


In [5]:
import os
import csv
import numpy as np
import tensorflow as tf
import pickle

import yaml
from tensorflow.keras import layers, activations, models, preprocessing, utils
from gensim.models.keyedvectors import  KeyedVectors

import re
from matplotlib import pyplot


print("tensor version {}".format(tf.version.VERSION))

dir_path = "/content/drive/My Drive/"
files_list = os.listdir(dir_path + os.sep)
ruta_glove = "/content/drive/My Drive/fasttext-sbwc.vec.gz"

filtros = '!"#$%&()*+-/<=>?@[\\]^_`{|}~\t\n'

questions = list()
resp = list()
answers = list()
vocab = []
input_characters = set()
target_characters = set()
print("Archivos a entrenar: {}".format(files_list))
for filepath in files_list:
    if filepath.endswith(".yaml"):
        stream = open(dir_path + os.sep + filepath, "rb")
        docs = yaml.safe_load(stream)
        conversations = docs["conversaciones"]
        for con in conversations:
            input_text = con[0]
            if len(con) > 2:
                pregunta = input_text
                pregunta = pregunta.lower()
                # pregunta = re.sub("[^a-zA-Z]", " ", pregunta)
                questions.append(pregunta)
                res = con[1:]

                ans = ""
                for rep in res:
                    r = str(rep).lower()
                    # r = re.sub("[^a-zA-Z]", " ", rep)
                    ans += " " + r
                resp.append(ans)
            elif len(con) > 1:
                questions.append(input_text)
                target_text = con[1]
                resp.append(target_text)

                for char in separar_oracion_tokens(input_text):
                    if char not in input_characters:
                        input_characters.add(char)
                for char in separar_oracion_tokens(target_text):
                    if char not in target_characters:
                        target_characters.add(char)


answers_with_tags = list()
for i in range(len(resp)):
    if type(resp[i]) == str:
        answers_with_tags.append(resp[i])
    else:
        questions.pop(i)

for i in range(len(answers_with_tags)):
    answers.append("<START> " + answers_with_tags[i] + " <END>")

palabras = set()
palabras.add('start')
palabras.add('end')
palabras = palabras.union(target_characters)
palabras = palabras.union(input_characters)
palabras = sorted(list(palabras))
tokenizer = preprocessing.text.Tokenizer(filters=filtros)
tokenizer.fit_on_texts(palabras)
VOCAB_SIZE = len(tokenizer.word_index) + 1
print("vocab size : {}".format(VOCAB_SIZE))

for word in tokenizer.word_index:
    vocab.append(word)




# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(x) for x in tokenized_questions])
padded_questions = preprocessing.sequence.pad_sequences(
    tokenized_questions, maxlen=maxlen_questions, padding="post"
)
encoder_input_data = np.array(padded_questions)
print(encoder_input_data.shape, maxlen_questions)

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = max([len(x) for x in tokenized_answers])
padded_answers = preprocessing.sequence.pad_sequences(
    tokenized_answers, maxlen=maxlen_answers, padding="post"
)
decoder_input_data = np.array(padded_answers)
print(decoder_input_data.shape, maxlen_answers)

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences(answers)
for i in range(len(tokenized_answers)):
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences(
    tokenized_answers, maxlen=maxlen_answers, padding="post"
)
onehot_answers = utils.to_categorical(padded_answers, VOCAB_SIZE)
decoder_output_data = np.array(onehot_answers)
print(decoder_output_data.shape)

# Saving all the arrays to storage
np.save("enc_in_data.npy", encoder_input_data)
np.save("dec_in_data.npy", decoder_input_data)
np.save("dec_tar_data.npy", decoder_output_data)

word_vectors = KeyedVectors.load_word2vec_format(ruta_glove)

embedding_matrix = np.zeros((VOCAB_SIZE, 300))

for i, word in enumerate(vocab):
  try:
    embedding_vector = word_vectors.get_vector(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector
  except KeyError as e:
    print('No esta en glove: {}'.format(e))   
# embeddings_index = {}
# with open('glove.6B.50d.txt', encoding='utf-8') as f:
#     for line in f:
#         values = line.split()
#         word = values[0]
#         coefs = np.asarray(values[1:], dtype='float32')
#         embeddings_index[word] = coefs
#     f.close()

# print("Glove Loded!")



encoder_inputs = tf.keras.layers.Input(shape=(None,))
embedding_layer_in = tf.keras.layers.Embedding( input_dim=VOCAB_SIZE, output_dim=300, weights=[embedding_matrix], mask_zero=True)
encoder_embedding = embedding_layer_in(
    encoder_inputs
)
# encoder_embedding.set_weights([embedding_matrix])

# encoder_embedding.set_weights([])
encoder_outputs, state_h, state_c = tf.keras.layers.LSTM(300, return_state=True)(
    encoder_embedding
)
encoder_states = [state_h, state_c]

decoder_inputs = tf.keras.layers.Input(shape=(None,))
embedding_layer_de = tf.keras.layers.Embedding( input_dim=VOCAB_SIZE, output_dim=300, weights=[embedding_matrix], mask_zero=True)
decoder_embedding = embedding_layer_de(
    decoder_inputs
)

decoder_lstm = tf.keras.layers.LSTM(300, return_state=True, return_sequences=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(
    VOCAB_SIZE, activation=tf.keras.activations.softmax
)
output = decoder_dense(decoder_outputs)

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output)
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)



tensor version 2.2.0-rc2
Archivos a entrenar: ['Google Fotos', "Apk's & Games", 'Documents', 'Enpass', 'MyBoy', 'Bluecoins', 'Untitled (2)', 'Classroom', 'Análisis de Data', 'Redes', 'Tareas', 'arc-history-export.json', 'Copy of StarWarsRacerV2_2018.zip', 'raspberry', 'Legaldo Privacy Policy.gdoc', 'Glosario 1-Vacio.gdoc', 'Guia de Lectura para el Primer Parcial HIP.gdoc', 'LEGALBOT.docx.gdoc', 'Glosario 2do Parcial-Linea.gdoc', 'Untitled document.gdoc', 'Registros Weintek-Swedish.gsheet', 'Anyfile Notepad Files', 'Estadisticas.xlsx', 'app.barcam.org.gdoc', 'fxlauncher', 'Untitled (1)', 'Untitled', 'Colab Notebooks', 'Swedish peticiones.gdoc', 'Sistema de facturacion.drawio', 'bot.yaml', 'variaciones2.yaml', 'variaciones1.yaml', 'preguntas.yaml', 'model.h5', 'Proyecto final', 'fasttext-sbwc.300k.vec.gz', 'fasttext-sbwc.vec.gz']
vocab size : 2915
(1151, 38) 38
(1151, 321) 321
(1151, 321, 2915)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


No esta en glove: "word 'ct.' not in vocabulary"
No esta en glove: "word ';' not in vocabulary"
No esta en glove: "word '100' not in vocabulary"
No esta en glove: "word '35' not in vocabulary"
No esta en glove: "word '38' not in vocabulary"
No esta en glove: "word '91' not in vocabulary"
No esta en glove: "word '8' not in vocabulary"
No esta en glove: "word 'empleador.' not in vocabulary"
No esta en glove: "word 'etc.' not in vocabulary"
No esta en glove: "word 'siguiente:' not in vocabulary"
No esta en glove: "word 'trabajador.' not in vocabulary"
No esta en glove: "word ',' not in vocabulary"
No esta en glove: "word '.' not in vocabulary"
No esta en glove: "word '..' not in vocabulary"
No esta en glove: "word '02' not in vocabulary"
No esta en glove: "word '04' not in vocabulary"
No esta en glove: "word '05' not in vocabulary"
No esta en glove: "word '07' not in vocabulary"
No esta en glove: "word '08' not in vocabulary"
No esta en glove: "word '1' not in vocabulary"
No esta en glove

In [1]:
model.summary()

history = model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_output_data,
    validation_split=0.2,
    verbose=1,
    batch_size=32,
    epochs=110,
)

# print(history.history['loss'])
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

pyplot.plot(history.history['val_accuracy'])
pyplot.plot(history.history['accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('accuracy')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

model.save("model.h5")


def make_inference_models():
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)

    decoder_state_input_h = tf.keras.layers.Input(shape=(300,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(300,))

    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding, initial_state=decoder_states_inputs
    )
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
    )

    return encoder_model, decoder_model


def str_to_tokens(sentence: str):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append(tokenizer.word_index[word])
    return preprocessing.sequence.pad_sequences(
        [tokens_list], maxlen=maxlen_questions, padding="post"
    )


enc_model, dec_model = make_inference_models()

for _ in range(10000):
    states_values = enc_model.predict(str_to_tokens(input("Enter question : ")))
    empty_target_seq = np.zeros((1, 1))
    empty_target_seq[0, 0] = tokenizer.word_index["start"]
    stop_condition = False
    decoded_translation = ""
    while not stop_condition:
        dec_outputs, h, c = dec_model.predict([empty_target_seq] + states_values)
        sampled_word_index = np.argmax(dec_outputs[0, -1, :])
        sampled_word = None
        for word, index in tokenizer.word_index.items():
            if sampled_word_index == index:
                decoded_translation += " {}".format(word)
                sampled_word = word

        if sampled_word == "end":
            stop_condition = True

        empty_target_seq = np.zeros((1, 1))
        empty_target_seq[0, 0] = sampled_word_index
        states_values = [h, c]

    print(decoded_translation)

NameError: ignored